<a href="https://colab.research.google.com/github/yoursdream123/banoqabil6-8/blob/main/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain pinecone-client google-generativeai openai tqdm python-dotenv -q
!pip install langchain-google-genai -U -q
!pip install --upgrade langchain -q
!pip install langchain-community -U -q
!pip install -qU langchain-google-vertexai
!pip install google-generativeai --upgrade -q
!pip install langchain-google-vertexai --upgrade -q
!pip install google-cloud google-cloud-language google-auth google-auth-oauthlib google-auth-httplib2 -q
!pip install -U langchain-google-vertexai -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.4/90.4 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.9/434.9 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 67.4 MB/s eta 0:00:00


In [29]:
!pip install google-generativeai --upgrade

In [32]:
!pip show google-generativeai

Name: google-generativeai
Version: 0.8.3
Summary: Google Generative AI High level API client library and tools.
Home-page: https://github.com/google/generative-ai-python
Author: Google LLC
Author-email: googleapis-packages@google.com
License: Apache 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: google-ai-generativelanguage, google-api-core, google-api-python-client, google-auth, protobuf, pydantic, tqdm, typing-extensions
Required-by: langchain-google-genai


In [14]:
with open(".env", "w") as f:
    f.write("pcsk_2Zm5Ym_PttGgRU6SikpLfXQAqvAvDs2TiQpPBLKoUH8sE8qFPyaySpxeteC3LGDesCSopc")
    f.write("us-east-1")
    f.write("GOOGLE_API_KEY")
print(".env file created successfully!")

.env file created successfully!


In [15]:
import os
from dotenv import load_dotenv

load_dotenv()
PINECONE_API_KEY = os.getenv("pcsk_2Zm5Ym_PttGgRU6SikpLfXQAqvAvDs2TiQpPBLKoUH8sE8qFPyaySpxeteC3LGDesCSopc")
PINECONE_ENVIRONMENT = os.getenv("us-east-1")
GOOGLE_GEMINI_API_KEY = os.getenv("GOOGLE_API_KEY")

In [16]:
import pinecone
from pinecone import Pinecone

# Instead of using pinecone.init(), create a Pinecone instance:
pc = Pinecone(api_key="pcsk_2Zm5Ym_PttGgRU6SikpLfXQAqvAvDs2TiQpPBLKoUH8sE8qFPyaySpxeteC3LGDesCSopc", environment="us-east-1")

index_name = "gemini-rag-index"
# Call list_indexes() on the Pinecone instance 'pc'
if index_name not in pc.list_indexes().names():  # Use pc.list_indexes().names() to get the index names
    pinecone.create_index(index_name, dimension=768)

index = pc.Index(index_name)
# Now you can use 'pc' to interact with Pinecone:
print(pc.list_indexes())

{'indexes': [{'deletion_protection': 'disabled',
              'dimension': 768,
              'host': 'gemini-rag-index-55zyeq9.svc.aped-4627-b74a.pinecone.io',
              'metric': 'cosine',
              'name': 'gemini-rag-index',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
              'status': {'ready': True, 'state': 'Ready'}}]}


In [8]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Load the documents, specifying the correct encoding
loader = TextLoader("/content/drive/MyDrive/MachineLearning.txt", encoding='latin-1')  # Try 'latin-1' or 'cp1252'
documents = loader.load()

# Split the documents into smaller chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.split_documents(documents)

print(f"Split into {len(docs)} chunks.")

Split into 1 chunks.


In [2]:
!pip install langchain-google-genai -U -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.0 MB/s eta 0:00:00


In [3]:
!pip install langchain-google-vertexai -U -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.4/90.4 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.9/434.9 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 43.5 MB/s eta 0:00:00


In [5]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.12
    Uninstalling langchain-0.3.12:
      Successfully uninstalled langchain-0.3.12


In [6]:
from langchain_google_genai.embeddings import GoogleGenerativeAIEmbeddings
from langchain_google_genai.chat_models import ChatGoogleGenerativeAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.schema import Document
import time

In [9]:
# Initialize the embedding model
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-004", api_key="GOOGLE_API_KEY")

# Upsert vectors into Pinecone
from tqdm import tqdm

# Upsert vectors into Pinecone
for i, doc in enumerate(tqdm(docs)):
    vector = embeddings.embed_query(doc.page_content)
    index.upsert([(str(i), vector, {"text": doc.page_content})])
    time.sleep(2)  # Introduce a 1-second delay between requests
print("Vectors upserted into Pinecone.")

print("Documents uploaded to Pinecone.")

  0%|          | 0/1 [00:00<?, ?it/s]ERROR:grpc._plugin_wrapping:AuthMetadataPluginCallback "<google.auth.transport.grpc.AuthMetadataPlugin object at 0x7bbb6330ed40>" raised exception!
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/google/auth/compute_engine/credentials.py", line 128, in refresh
    self._retrieve_info(request)
  File "/usr/local/lib/python3.10/dist-packages/google/auth/compute_engine/credentials.py", line 101, in _retrieve_info
    info = _metadata.get_service_account_info(
  File "/usr/local/lib/python3.10/dist-packages/google/auth/compute_engine/_metadata.py", line 323, in get_service_account_info
    return get(request, path, params={"recursive": "true"})
  File "/usr/local/lib/python3.10/dist-packages/google/auth/compute_engine/_metadata.py", line 248, in get
    raise exceptions.TransportError(
google.auth.exceptions.TransportError: ("Failed to retrieve http://metadata.google.internal/computeMetadata/v1/instance/service-account

GoogleGenerativeAIError: Error embedding content: Timeout of 60.0s exceeded, last exception: 503 Getting metadata from plugin failed with error: ("Failed to retrieve http://metadata.google.internal/computeMetadata/v1/instance/service-accounts/default/?recursive=true from the Google Compute Engine metadata service. Status: 404 Response:\nb''", <google.auth.transport.requests._Response object at 0x7bbb635e5330>)

bhai Abdul hakeem wala yahn say start hai

In [10]:
%pip install -qU langchain-pinecone langchain google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 5.7 MB/s eta 0:00:00


In [19]:
from pinecone import Pinecone, ServerlessSpec
from google.colab import userdata
pinecone_api_key = userdata.get("PINECONE_KEY")

pc = Pinecone(api_key="PINECONE_KEY")

In [23]:
index_name = "ragstart"

pc.create_index(
    name=index_name,
    dimension=768,
    metric="cosine",

spec=ServerlessSpec(cloud="aws",region="us-east-1"),
)
index = pc.index(index_name)

UnauthorizedException: (401)
Reason: Unauthorized
HTTP response headers: HTTPHeaderDict({'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': 'd2bdd66b286a46a49134ae07d5343c51', 'Date': 'Fri, 03 Jan 2025 18:42:50 GMT', 'Content-Type': 'text/html', 'Server': 'Google Frontend', 'Content-Length': '15', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: Invalid API Key
